# core

> Module for the least squares model fit.

In [ ]:
#| default_exp core

In [ ]:
#| hide
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import t
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


Create an instance of the class `CalibrationModel` by passing in the path to both your calibration data and your sample data. Next specify the name of your response variable and the nummber of test replicates you measured.

In [ ]:
sns.set_style("white")
sns.set_style("ticks")

In [ ]:
#| export

class CalibrationModel:
    def __init__(self, data, response_variable, test_replicates):
        self.raw_data = self.load_data(data)
        self.response_variable = response_variable
        self.fit = None
        self.test_replicates = test_replicates
        self.cal_line_points = self.raw_data.shape[0]
        self.r2 = None

    def load_data(self, data):
        if ".csv" in data:
            raw_data = pd.read_csv(data)
        else:
            raw_data = data
        return raw_data

    def fit_ols(self):
        X = self.raw_data[["concentration"]]
        y = self.raw_data[self.response_variable]
        lr = LinearRegression
        self.fit = lr().fit(X, y)
        self.r2 = self.fit.score(X, y)
        return self.fit

    def get_params(self):
        slope = self.fit.coef_[0]
        intercept = self.fit.intercept_
        return slope, intercept

    def get_r2(self):
        return self.r2

    def inverse_prediction(self, unknown: float):
        slope, intercept = self.get_params()
        return (unknown - intercept) / slope

    def calculate_sse(self):
        y_pred = self.fit.predict(self.raw_data[["concentration"]])
        return np.sum((y_pred - self.raw_data[self.response_variable]) ** 2)

    def calculate_syx(self):
        return np.sqrt((self.calculate_sse()) / (len(self.raw_data) - 2))

    def get_t_value(self, alpha):
        return t.ppf(1 - alpha / 2, len(self.raw_data) - 2)

    def calculate_uncertainty(self):
        return self.calculate_sxhat() * self.get_t_value(0.05)

    def calculate_sxhat(self):
        return (self.calculate_syx() / self.fit.coef_[0]) * np.sqrt(1 / self.test_replicates + self.cal_line_points)

    def fit_model(self):
        self.fit_ols()
        self.get_params()
        self.get_r2()
        self.calculate_uncertainty()
        self.tabulate_results()

    def plot_fit(self):
        sns.regplot(x="concentration", y=self.response_variable, data=self.raw_data)
        sns.despine()
        sns.set_context("paper")
        plt.title(f"Calibration curve of concentration versus {self.response_variable}")
        plt.xlabel("Concentration")
        plt.ylabel("Peak Value")
        plt.annotate(f"R-squared = {self.get_r2():.3f}", xy=(0.3, 0.8), xycoords="axes fraction", fontsize=9,
                     ha="center", va="center")
        plt.annotate(f"Regression formula: y = {self.get_params()[0]:.3f} * x + {self.get_params()[1]:.3f}",
                     xy=(0.3, 0.7), xycoords="axes fraction", fontsize=9, ha="center", va="center")
        plt.show()



    def tabulate_results(self):
        print(f"Calibration curve of {self.response_variable} versus concentration")
        print(f"R2 = {self.r2}")
        print(f"Slope = {self.get_params()[0]}")
        print(f"Intercept = {self.get_params()[1]}")
        print(f"Uncertainity = {self.calculate_uncertainty()}")
        # print(f"Prediction = {self.inverse_prediction()}")
    

In [ ]:
def generate_test_data(slope, intercept):
        x = np.linspace(1, 10, num=5)
        y = intercept + x * slope
        df = pd.DataFrame({'concentration': x, "abs": y})
        return df

def generate_sample_data():
    x = np.array(['unknown1', 'unknown2'])
    y = np.array([13.75, 20.50])
    df = pd.DataFrame({'sample': x, "abs": y})
    df = df.set_index('sample')
    return df


test_data = generate_test_data(3, 4)
sample_data = generate_sample_data()



In [ ]:
cal = CalibrationModel(data = test_data, response_variable = "abs", test_replicates = 1)
cal.fit_model()

Calibration curve of abs versus concentration
R2 = 1.0
Slope = 3.0000000000000004
Intercept = 3.9999999999999964
Uncertainity = 5.9589541336214675e-15


## Tests

A couple of quick tests to make sure nothing is broken. 

In [ ]:
assert cal.inverse_prediction(20.5) == 5.5

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()